In [40]:

import pandas as pd
import numpy as np
import regex as re
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

Last Scraped date for the data August 25th 2019

In [2]:
amazon = pd.read_csv('datasets/all_products.csv')

In [3]:
target = pd.read_csv('datasets/target.csv')

### Viewing the Amazon dataframe
***

In [95]:
a

'Banana Conventional Whole Trade Guarantee, 1 Each (Brands May Vary) '

In [94]:
a= amazon['all_info_combined'][1500]

In [86]:
a = re.sub(pattern="[?^*\n$]",repl=" ",string= a)
a = re.sub("[?^®]",'',a)

In [90]:
text

'Penne Pasta - 16oz - Barilla Serving Size: 2 oz Serving Per Container: ABOUT 8 Amount Per Serving Calories: 200 % Daily Value  Calcium 0% Calories From Fat 10Cal Cholesterol 0mg 0% Dietary Fiber 2g 8% Folic Acid 30% Insoluble Fiber 1g Iron 10% Protein 7g Saturated Fat 0g 0% Sodium 0mg 0% Soluble Fiber 1g Sugars 2g Total Carbohydrate 42g 14% Total Fat 1g 2% Trans Fat 0g Vitamin A 0% Vitamin B1 (Thiamin) 35% Vitamin B2 (Riboflavin) 15% Vitamin B3 (Niacin) 15% Vitamin C 0%   Percentage of Daily Values are based on a 2,000 calorie diet.'

In [97]:
def cleanser(text):
    # 1.Removing the \n characters from the text
    newline = re.sub(pattern="[?^*\n$]",repl=" ",string= text)
    # 2.Removing the special characters from the text
    special = re.sub('/[^a-zA-Z0-9 ]/g', "", newline);
    # 3.Lower casing the text
    lower_case = special.lower()
    # 4. Tokenizing based on space
    
    tokenizer = RegexpTokenizer('\s+', gaps=True)
    words = tokenizer.tokenize(lower_case)


    # 5. Removing the stop words
    stopwords_list = stopwords.words('english')
    # Adding some of the words that may leak data into our model
    stopwords_list.extend(['important','each', 'ingredients','serving','size','calorie','daily','values','diet','food'])
    stops = set(stopwords_list)
    words_nostop = [w for w in words if w not in stops]


    return (" ".join(words_nostop))

In [99]:
amazon['all_info_combined_clean'] = amazon['all_info_combined'].apply(cleanser)

In [101]:
target['all_info_combined_clean'] = target['all_info_combined'].apply(cleanser)

In [103]:
target.head()

,brand,category,name,description,price,store,all_info_combined,all_info_combined_clean
0,Ready Pac Foods,Produce,Ready Pac Foods Baby Spinach - 5.25oz,,2.99,Target,Ready Pac Foods Baby Spinach - 5.25oz,ready pac foods baby spinach - 5.25oz
1,Ready Pac Foods,Produce,Ready Pac Classic Caesar Salad Kit - 10oz,Serving Size: 2 cup\nServing Per Container: AB...,3.49,Target,Ready Pac Classic Caesar Salad Kit - 10oz Serv...,ready pac classic caesar salad kit - 10oz size...
2,Taylor Farms,Produce,Taylor Farms Chopped Kale Juicing Greens - 16oz,Serving Size: 2 cups\nServing Per Container: a...,3.69,Target,Taylor Farms Chopped Kale Juicing Greens - 16o...,taylor farms chopped kale juicing greens - 16o...
3,Eat Smart,Produce,Eat Smart Sweet Kale Vegetable Salad Kit - 12oz,Serving Size: 1 cup\nServing Per Container: 4\...,3.99,Target,Eat Smart Sweet Kale Vegetable Salad Kit - 12o...,eat smart sweet kale vegetable salad kit - 12o...
4,Ready Pac Foods,Produce,Ready Pac Foods Shredded Iceberg Lettuce - 8oz,Serving Size: 1.5 cup\nServing Per Container: ...,1.79,Target,Ready Pac Foods Shredded Iceberg Lettuce - 8oz...,ready pac foods shredded iceberg lettuce - 8oz...


In [4]:
amazon.shape

(10714, 7)

In [5]:
amazon.category.unique()

array(['Dairy', 'Produce', 'Snacks & Sweets', 'Frozen Foods',
       'Breakfast & Cereals', 'Beverages', 'Coffee & Tea',
       'Sports & Energy Drinks', 'Pantry', 'Bread & Bakery',
       'Meat & Seafood'], dtype=object)

In [6]:
amazon.tail(2)

,brand,category,description,important_info,name,price,store
10712,365 Everyday Value,Dairy,Organic mild cheddar cheese shreds produced wi...,Important information\n\nIngredients\nCULTURED...,"365 Everyday Value, Organic Shredded Mild Ched...",6.99,Whole Foods Market
10713,Daiya,Dairy,Product Description\nWe focus on people's favo...,Important information\n\nIngredients\nFiltered...,"Daiya, Mozzarella Style Shreds, 8 oz",5.39,Whole Foods Market


In [7]:
amazon['name'] = amazon['name'].fillna('')
amazon['important_info'] = amazon['important_info'].fillna('')
amazon['description'] = amazon['description'].fillna('')


In [8]:
amazon['all_info_combined'] = amazon[['name','description']].apply(lambda x: ' '.join(x), axis=1)
# 'important_info'

In [104]:
amazon.isna().sum()

brand                        0
category                     0
description                  0
name                         0
price                      295
store                        0
all_info_combined            0
all_info_combined_clean      0
dtype: int64

In [11]:
# impute the rows that are missing the brand name with the first part of the name
# before the comma
for index, row in amazon.loc[amazon['brand'].isna()].iterrows():
    amazon.loc[index,'brand']=amazon.loc[index,'name'].split(',')[0]
  

In [12]:
for i in amazon.category.unique():
    print(f"Whole Foods Category {i} is out of stock for {amazon[(amazon.store=='Whole Foods Market') & (amazon.price.isna()) & (amazon.category ==i)].shape[0]} number of products")
          
          

Whole Foods Category Dairy is out of stock for 16 number of products
Whole Foods Category Produce is out of stock for 14 number of products
Whole Foods Category Snacks & Sweets is out of stock for 29 number of products
Whole Foods Category Frozen Foods is out of stock for 9 number of products
Whole Foods Category Breakfast & Cereals is out of stock for 12 number of products
Whole Foods Category Beverages is out of stock for 8 number of products
Whole Foods Category Coffee & Tea is out of stock for 8 number of products
Whole Foods Category Sports & Energy Drinks is out of stock for 0 number of products
Whole Foods Category Pantry is out of stock for 1 number of products
Whole Foods Category Bread & Bakery is out of stock for 3 number of products
Whole Foods Category Meat & Seafood is out of stock for 16 number of products


In [13]:
# Only keep the digits and remove any extra characters in the price column
amazon['price'] = amazon.price.str.extract(r'(\d+\.\d+)').astype(float)

In [14]:
amazon[amazon.price>200]

,brand,category,description,important_info,name,price,store,all_info_combined
4108,Keurig,Beverages,The Keurig K155 Office PRO Commercial Coffee M...,,Keurig K155 Office Pro Single Serve Commercial...,299.99,Amazon,Keurig K155 Office Pro Single Serve Commercial...


In [15]:
#Drop the out of stock products
amazon['price'].dropna(inplace=True)
amazon.price.isna().sum()
amazon.reset_index(drop=True)

,brand,category,description,important_info,name,price,store,all_info_combined
0,Organic Valley,Dairy,When cows spend their days on lush green pastu...,Important information\n\nIngredients\nOrganic ...,"Organic Valley, Organic Half & Half, Ultra Pas...",4.69,Whole Foods Market,"Organic Valley, Organic Half & Half, Ultra Pas..."
1,Organic Valley,Dairy,When cows spend their days on lush green pastu...,Important information\n\nIngredients\nOrganic ...,"Organic Valley, Organic Half & Half, Ultra Pas...",2.69,Amazon,"Organic Valley, Organic Half & Half, Ultra Pas..."
2,Organic Valley,Dairy,When cows spend their days on lush green pastu...,Important information\n\nIngredients\nOrganic ...,"Organic Valley, Organic Heavy Whipping Cream, ...",4.49,Amazon,"Organic Valley, Organic Heavy Whipping Cream, ..."
3,Folgers Gourmet Selections Lively Colombian Co...,Dairy,Folgers Gourmet Selections K-Cups - Med - Colu...,Important information\n\nLegal Disclaimer\nSta...,Folgers Gourmet Selections Lively Colombian Co...,9.06,Amazon,Folgers Gourmet Selections Lively Colombian Co...
4,Lactaid,Dairy,You won’t believe it’s 100% lactose free. Get ...,Important information\n\nSafety Warning\nDo no...,"Lactaid 2% Reduced Fat Milk, 64 fl oz",3.99,Amazon,"Lactaid 2% Reduced Fat Milk, 64 fl oz You won’..."
5,Folgers,Dairy,A select line of exceptional coffee blends car...,Important information\n\nSafety Warning\nThis ...,"Folgers French Roast Ground Coffee, Medium-Dar...",8.99,Amazon,"Folgers French Roast Ground Coffee, Medium-Dar..."
6,365 Everyday Value,Dairy,Coffee Creamer Vanilla,Important information\n\nIngredients\nNonfat M...,"365 Everyday Value Coffee Creamer Vanilla, 16 ...",2.79,Amazon,"365 Everyday Value Coffee Creamer Vanilla, 16 ..."
7,365 Everyday Value,Dairy,Real Dairy Whipped Cream,Important information\n\nIngredients\nORGANIC ...,"365 Everyday Value, Real Dairy Whipped Cream, ...",3.99,Amazon,"365 Everyday Value, Real Dairy Whipped Cream, ..."
8,DairyPure,Dairy,Fresh milk is ideal for the entire family. Ful...,Important information\n\nLegal Disclaimer\nSta...,DairyPure 2% Reduced Fat Milk - Half Gallon,2.39,Amazon,DairyPure 2% Reduced Fat Milk - Half Gallon Fr...
9,365 Everyday Value,Dairy,"There's no better way to ""go organic"" than wit...",Important information\n\nIngredients\nORGANIC ...,"365 Everyday Value, Organic Whole Milk, 128 oz",5.99,Whole Foods Market,"365 Everyday Value, Organic Whole Milk, 128 oz..."


In [16]:
# Ask Wes about this.
# amazon.loc[amazon['price']>10]

**Work on Target Dataset and get the columns in the right formats**

In [17]:
target.shape

(3196, 94)

In [18]:
target.head(2)

,Alcohol content,Baked good type,Bakery topping,Beef lean percent,Beverage container material,Bone cut type,Bread,CPSC Choking Hazard Warnings,Caffeine claim 1,Calories,...,Type of Meat,Type of Rice or Grain,UPC,Vinegar type,Water enhancement type 1,brand,category,labelinfo,name,price
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,7.745207e+10,NaN,NaN,Ready Pac Foods,Target/Grocery/Produce/Fresh Vegetables,NaN,Ready Pac Foods Baby Spinach - 5.25oz,2.99
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,150,...,NaN,NaN,7.774521e+10,NaN,NaN,Ready Pac Foods,Target/Grocery/Produce/Fresh Vegetables,Serving Size: 2 cup\nServing Per Container: AB...,Ready Pac Classic Caesar Salad Kit - 10oz,3.49


In [19]:
# Make a column that combines all of the countries of origin
target['Country of Origin'] = target['Country of Origin'].fillna('')
target['Country of origin 2'] = target['Country of origin 2'].fillna('')
target['Country of origin 3'] = target['Country of origin 3'].fillna('')
target['Origin'] = target['Origin'].fillna('')
target['country_of_origin'] = target[['Country of Origin','Country of origin 2', 'Country of origin 3','Origin']].apply(lambda x: ','.join(x), axis=1)
target['country_of_origin'].replace('','unknown', inplace=True)
# Remove all of the other origin columns
target.drop(columns=['Country of Origin','Country of origin 2','Country of origin 3','Origin'], inplace=True)

In [20]:
target = target[['brand','category','name', 'labelinfo','price']]

In [21]:
target['store'] = 'Target'

In [22]:
target['category'] = [x.split('/')[2] for x in target['category']]

In [23]:
target['category'].unique()

array(['Produce', 'Frozen Foods', 'Meat & Seafood', 'Beverages',
       'Specialty Diets ', 'Nutrition & Weight Loss', 'Dairy', 'Deli',
       'Chips, Snacks & Cookies', 'Bakery & Bread', 'Breakfast & Cereal',
       'Pantry', 'Candy'], dtype=object)

In [24]:
# Standardizing the category names with the Amazon dataset.
target.loc[target['category']=='Breakfast & Cereal'] = 'Breakfast & Cereals'
target.loc[(target['category']=='Chips, Snacks & Cookies') | (target['category']=='Candy')] = 'Snacks & Sweets'

In [25]:
amazon['category'].unique()

array(['Dairy', 'Produce', 'Snacks & Sweets', 'Frozen Foods',
       'Breakfast & Cereals', 'Beverages', 'Coffee & Tea',
       'Sports & Energy Drinks', 'Pantry', 'Bread & Bakery',
       'Meat & Seafood'], dtype=object)

In [26]:
weight_units = ['ABV', 'Ounce', 'oz', 'Proof', 'mL', 'Ml', 'ml', 'Fl Oz', 'Gallon', 'Pound','L', 'ct', 'CT','Liter', 'ea', 'lb']

In [27]:
target.rename(columns={'labelinfo':'description'},inplace=True)
# target.rename(columns={'price':'full_price'},inplace=True)

In [28]:
# Clean any white space that may be in the name
target['name'] = target['name'].str.strip()
target['description'] = target['description'].str.strip()

In [29]:
target['description'] = target['description'].fillna('')
target['all_info_combined'] = target[['name','description']].apply(lambda x: ' '.join(x), axis=1)

Last check before merging the two dataframes

In [105]:
amazon.head()

,brand,category,description,name,price,store,all_info_combined,all_info_combined_clean
0,Organic Valley,Dairy,When cows spend their days on lush green pastu...,"Organic Valley, Organic Half & Half, Ultra Pas...",4.69,Whole Foods Market,"Organic Valley, Organic Half & Half, Ultra Pas...","organic valley, organic half & half, ultra pas..."
1,Organic Valley,Dairy,When cows spend their days on lush green pastu...,"Organic Valley, Organic Half & Half, Ultra Pas...",2.69,Amazon,"Organic Valley, Organic Half & Half, Ultra Pas...","organic valley, organic half & half, ultra pas..."
2,Organic Valley,Dairy,When cows spend their days on lush green pastu...,"Organic Valley, Organic Heavy Whipping Cream, ...",4.49,Amazon,"Organic Valley, Organic Heavy Whipping Cream, ...","organic valley, organic heavy whipping cream, ..."
3,Folgers Gourmet Selections Lively Colombian Co...,Dairy,Folgers Gourmet Selections K-Cups - Med - Colu...,Folgers Gourmet Selections Lively Colombian Co...,9.06,Amazon,Folgers Gourmet Selections Lively Colombian Co...,folgers gourmet selections lively colombian co...
4,Lactaid,Dairy,You won’t believe it’s 100% lactose free. Get ...,"Lactaid 2% Reduced Fat Milk, 64 fl oz",3.99,Amazon,"Lactaid 2% Reduced Fat Milk, 64 fl oz You won’...","lactaid 2% reduced fat milk, 64 fl oz won’t be..."


In [106]:
target.head()

,brand,category,name,description,price,store,all_info_combined,all_info_combined_clean
0,Ready Pac Foods,Produce,Ready Pac Foods Baby Spinach - 5.25oz,,2.99,Target,Ready Pac Foods Baby Spinach - 5.25oz,ready pac foods baby spinach - 5.25oz
1,Ready Pac Foods,Produce,Ready Pac Classic Caesar Salad Kit - 10oz,Serving Size: 2 cup\nServing Per Container: AB...,3.49,Target,Ready Pac Classic Caesar Salad Kit - 10oz Serv...,ready pac classic caesar salad kit - 10oz size...
2,Taylor Farms,Produce,Taylor Farms Chopped Kale Juicing Greens - 16oz,Serving Size: 2 cups\nServing Per Container: a...,3.69,Target,Taylor Farms Chopped Kale Juicing Greens - 16o...,taylor farms chopped kale juicing greens - 16o...
3,Eat Smart,Produce,Eat Smart Sweet Kale Vegetable Salad Kit - 12oz,Serving Size: 1 cup\nServing Per Container: 4\...,3.99,Target,Eat Smart Sweet Kale Vegetable Salad Kit - 12o...,eat smart sweet kale vegetable salad kit - 12o...
4,Ready Pac Foods,Produce,Ready Pac Foods Shredded Iceberg Lettuce - 8oz,Serving Size: 1.5 cup\nServing Per Container: ...,1.79,Target,Ready Pac Foods Shredded Iceberg Lettuce - 8oz...,ready pac foods shredded iceberg lettuce - 8oz...


In [32]:
amazon.drop(columns = ['important_info'], inplace =True)

In [107]:
all_stores = pd.DataFrame()

In [108]:
all_stores = amazon.append(target)

/Users/valamani/anaconda3/envs/dsi/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [109]:
all_stores.tail()

,all_info_combined,all_info_combined_clean,brand,category,description,name,price,store
3191,Maxwell House French Vanilla Cafe Medium Roast...,maxwell house french vanilla cafe medium roast...,Maxwell House,Beverages,Serving Size: 1.33 tbsp\nServing Per Container...,Maxwell House French Vanilla Cafe Medium Roast...,3.89,Target
3192,Nescafé Taster's Choice House Blend Instant Co...,nescafé taster's choice house blend instant co...,Nescafe,Specialty Diets,,Nescafé Taster's Choice House Blend Instant Co...,1.29,Target
3193,Café Bustelo Espresso Dark Roast Instant Coffe...,café bustelo espresso dark roast instant coffe...,Cafe Bustelo,Beverages,,Café Bustelo Espresso Dark Roast Instant Coffe...,6.39,Target
3194,Nescafé Taster's Choice French Roast Medium Da...,nescafé taster's choice french roast medium da...,Nescafe,Beverages,,Nescafé Taster's Choice French Roast Medium Da...,8.59,Target
3195,Nescafé Taster's Choice House Blend Light Roas...,nescafé taster's choice house blend light roas...,Nescafe,Beverages,,Nescafé Taster's Choice House Blend Light Roas...,8.59,Target


In [110]:
all_stores.to_csv('datasets/comp_data3.csv',index=False)

In [37]:
# all_stores = all_stores.append(target[['Category','name','brand','store','full_price','brand_name_weight']])
# all_stores.drop(['weight', 'sale_price'], axis =1)

Building a basic recommender system

In [38]:
# test1 =['Bristol Farms', ' Beet Salad']
# test2 = ['PL', ' Pl Herb Salad', ' 5 Ounce']
# test3 = ['Croissant Tuna Salad', ' 11 Ounce']
# def clean(brand_name_weight):
#     contains_weigth = False
#     for i in brand_name_weight:
# #         if bool(re.findall('[\d]+',i)):
#         if bool(re.findall( r'[0-9]+',i)):
#             contains_weight = True
            
#     if len(brand_name_weight) == 1:
#         name = brand_name_weight[0]
#     if len(brand_name_weight) == 2 and contains_num:
#         name = brand_name_weight[0]
#         weight = brand_name_weight[1]
#     elif len(brand_name_weight) == 2 and not contains_num:
#         brand = brand_name_weight[0]
#         name  = brand_name_weight[1]
#     elif len(brand_name_weight) == 3:
#         brand = brand_name_weight[0]
#         name = brand_name_weight[1]
#         weight = brand_name_weight[2]
#     elif len
        
#     return contains_num

        



# # Formatting the category names for Bristol Farms
# # Alcoholic Beverage
# bristol['Category'].loc[bristol.Category =='Beer']  = 'Alcoholic Beverage'
# bristol['Category'].loc[bristol.Category =='Red Wine']  = 'Alcoholic Beverage'  
# bristol['Category'].loc[bristol.Category =='White Wine']  = 'Alcoholic Beverage'
# bristol['Category'].loc[bristol.Category =='Spirit']  = 'Alcoholic Beverage' 
# # For the produce
# bristol['Category'].loc[bristol.Category =='Fresh Fruits']  = 'Produce'
# bristol['Category'].loc[bristol.Category =='Fresh Vegetable']  = 'Produce'
# # For Meat & Sea Food
# bristol['Category'].loc[bristol.Category =='Beef']  = 'Meat & Seafood'
# bristol['Category'].loc[bristol.Category =='Chicken']  = 'Meat & Seafood'
# bristol['Category'].loc[bristol.Category =='Lamb']  = 'Meat & Seafood'
# bristol['Category'].loc[bristol.Category =='Pork']  = 'Meat & Seafood'
# bristol['Category'].loc[bristol.Category =='Fish']  = 'Meat & Seafood'
# bristol['Category'].loc[bristol.Category =='Shellfis']  = 'Meat & Seafood'
# # For Prepared food
# bristol['Category'].loc[bristol.Category =='Salads']  = 'Prepared food'
# bristol['Category'].loc[bristol.Category =='Soups']  = 'Prepared food'
# bristol['Category'].loc[bristol.Category =='Entrees']  = 'Prepared food'
# bristol['Category'].loc[bristol.Category =='Sandwiche']  = 'Prepared food'
# # For Bread & Bakery
# bristol['Category'].loc[bristol.Category =='Cakes'] = 'Bread & Bakery'
# bristol['Category'].loc[bristol.Category =='Dessert'] = 'Bread & Bakery'
# # For Snacks
# bristol['Category'].loc[bristol.Category =='Snack Foods'] = 'Snacks'



# # Formatting the category names for wf Farms
# # Alcoholic Beverage
# wf['Category'].loc[wf.Category ==' Wine & Beer ']  = 'Alcoholic Beverage'
 
# # For the produce
# wf['Category'].loc[wf.Category ==' Produce & Organic Produce ']  = 'Produce'
# wf['Category'].loc[wf.Category =='Fresh Vegetable']  = 'Produce'
# # For Meat & Sea Food
# wf['Category'].loc[wf.Category ==' Seafood ']  = 'Meat & Seafood'
# wf['Category'].loc[wf.Category ==' Beef, Poultry, Pork, & More ']  = 'Meat & Seafood'
# # For Snacks
# wf['Category'].loc[wf.Category ==' Snacks, Chips, Salsas, & Dips ']  = 'Snacks'
# # Prepared Food
# wf['Category'].loc[wf.Category ==' Prepared Foods ']  = 'Prepared food'
# # Dairy & Eff
# wf['Category'].loc[wf.Category ==' Cheese '] = 'Dairy & Egg'
# wf['Category'].loc[wf.Category ==' Dairy & Eggs '] = 'Dairy & Egg'
# # Bread & Bakery
# wf['Category'].loc[wf.Category ==' Breads, Rolls, & Bakery '] = 'Bread & Bakery'
# # Strip the spaces that are in each of the rows.
# wf['Category'] =[x.strip() for x in wf['Category']]
